# Advanced Certification Program in Computational Data Science

##  A program by IISc and TalentSprint

### Mini Project Notebook 1 : Data analytics



## Learning Objectives



At the end of the experiment, you will be able to :


* understand the requirements for a “clean” dataset, ready for use in statistical analysis

* use Python libraries like Pandas, Numpy, and Matplotlib to perform the  data-preprocessing steps

* obtain probability and statistics based insights from the data


## Dataset

The dataset chosen for this experiment is the **Play Store** dataset which is  publicly available and created with this [methodology](https://nycdatascience.com/blog/student-works/google-play-store-everything-that-you-need-to-know-about-the-android-market/)  

This dataset consists of 10841 records. Each record is made up of 13 fields.

**For example**, Each record consists of App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, and Android Ver.

## Problem Statement

Before we can derive any meaningful insights from the Play Store data, it is essential to pre-process the data and make it suitable for further analysis. This pre-processing step forms a major part of data wrangling (or data munging) and ensures better quality data. It consists of the transformation and mapping of data from a "raw" data form into another format so that it is more valuable for a variety of downstream purposes such as analytics. Data analysts typically spend a sizeable amount of time in the process of data wrangling, compared to the actual analysis of the data.

After data munging is performed, several actionable insights can be derived from the Play Store apps data. Such insights could help to unlock the enormous potential to drive app-making businesses to success.

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/googleplaystore.csv

#### Import required packages

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import zscore
import matplotlib.ticker as ticker
from scipy import stats
from scipy.stats import ttest_ind

#### Load the dataset

In [ ]:
# YOUR CODE HERE
df_ = pd.read_csv('https://cdn.iisc.talentsprint.com/CDS/Datasets/googleplaystore.csv')
df = df_.copy()
print(df.head(5))


## Pre-processing

### Task 1: Data Cleaning

* Check whether there are any null values and figure out how you want to handle them?
  
    **Hint:** isnan(), dropna(), fillna()
* If there is any duplication of a record, how would you like to handle it?

    Hint: [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

* Are there any non-English apps? And how to filter them?

* In the size column, multiply 1,000,000 with M in the cell and multiply by 1000 if we have K in the cell.

In [ ]:
#YOUR CODE HERE
null_values = df_.isna().sum()
print("all columns with null\n\n",null_values)

#fill 0.0 for all null values of rating
df['Rating'].fillna(0.0, inplace=True)

# if price is 0, update type as free else paid.
df['Type'] = np.where(df['Price'] == '0', 'Free', 'Paid')
df.drop(df[df['Type']=='0'].index,inplace=True)


#If Content Rating is nan, drop the record.
df.dropna(subset=['Content Rating'], inplace=True)

#if current ver is nan, fill it with the most occuring value
df['Current Ver'].fillna(df_['Current Ver'].mode()[0], inplace=True)


#if Android ver is nan, fill it with the most occuring value
df['Android Ver'].fillna(df_['Android Ver'].mode()[0], inplace=True)

#
df['Installs'] = df['Installs'].str.replace(',', '').str.rstrip('+').astype(int)


In [ ]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#non english apps
df = df[df['App'].apply(lambda x: all(ord(char) < 128 for char in x))]
df.shape

In [ ]:
convert_size_to_numeric = lambda size_str: (
    'Varies with device' if 'Varies with device' in size_str
    else float(size_str.replace('M', '')) * 1_000_000 if 'M' in size_str
    else float(size_str.replace('K', '')) * 1_000 if 'K' in size_str
    else None
)

df['Size'] = df['Size'].apply(lambda x: convert_size_to_numeric(x) if pd.notna(x) else None)


## Visualization

### Task 2: Perform the  following tasks:

##### Exercise 1: Find the number of apps in various categories by using an appropriate plot.

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(12, 8))
sns.countplot(y='Category', data=df, order=df['Category'].value_counts().index, palette='viridis')
plt.title('Number of Apps in Various Categories')
plt.xlabel('Number of Apps')
plt.ylabel('Category')
plt.show()

##### Exercise 2: Explore the distribution of free and paid apps across different categories

**Hint:** Stacked Bar Graph, [link](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/bar_stacked.html)

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(14, 8))

# Pivot the DataFrame to create a table for the stacked bar plot
stacked_df = df.groupby(['Category', 'Type']).size().unstack()
#sort by numbers of apps

# Create a stacked bar plot
stacked_df.plot(kind='bar', stacked=True, color=['green', 'red'], edgecolor='white', figsize=(14, 8))

plt.title('Distribution of Free and Paid Apps Across Categories')
plt.xlabel('Category')
plt.ylabel('Number of Apps')

# Add a legend for clarity
plt.legend(title='App Type', loc='upper right')

plt.xticks(rotation=90)  # Rotate x-axis labels for better readability

plt.show()


##### Exercise 3: Represent the distribution of app rating on a scale of 1-5 using an appropriate plot

**Hint:** histogram / strip plot

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(14, 6))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['Rating'], bins=30, kde=True, color='skyblue')
plt.title('Histogram of App Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequency')

# Strip Plot
plt.subplot(1, 2, 2)
sns.stripplot(x='Rating', data=df, jitter=True, alpha=0.7, color='salmon')
plt.title('Strip Plot of App Ratings')
plt.xlabel('Rating')
plt.ylabel('')

plt.tight_layout()
plt.show()


##### Exercise 4: Identify outliers of the rating column by plotting the boxplot category wise and handle them.

**Hint:** Removing outliers using Z-score, quantile [link](https://kanoki.org/2020/04/23/how-to-remove-outliers-in-python/)

In [ ]:
# YOUR CODE HERE


# Identify and handle outliers using Z-score
df['Z_Score'] = zscore(df['Rating'])

mean_z = np.mean(df['Z_Score'])
std_z = np.std(df['Z_Score'])

# Choose the multiplier based on the desired threshold (e.g., 2 for 95% confidence interval)
threshold_multiplier = 2

# Calculate the threshold
threshold = mean_z + threshold_multiplier * std_z

selected_column = df['Z_Score']

# Create a new DataFrame with the selected column
selected_column_df = pd.DataFrame({'Z_Score': selected_column})

# Save the new DataFrame to a CSV file
selected_column_df.to_csv('Z_Score.csv', index=False)

outliers = np.abs(df['Z_Score']) > threshold

df_filtered=df[~outliers]

plt.figure(figsize=(16, 8))

# Create a boxplot
sns.boxplot(x='Category', y='Rating', data=df_filtered)
plt.title('Boxplot of App Ratings by Category')
plt.xlabel('Category')
plt.ylabel('Rating')
plt.xticks(rotation=90)
plt.show()

##### Exercise 5: Plot the barplot of all the categories indicating no. of installs

In [ ]:
# YOUR CODE HERE
# Assuming your DataFrame is named df
# Create a bar plot using seaborn

plt.figure(figsize=(14, 8))
sns.barplot(x='Installs', y='Category', data=df, estimator=sum, errorbar=None)

# Add labels and title
plt.title('Number of Installs by Category')
plt.xlabel('Number of Installs')
plt.ylabel('Category')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)
plt.show()


## Insights


### Task 3: Derive the below insights

##### Exercise 1: Does the price correlate with the size of the app?

  **Hint:** plot the scatterplot of `Size` and `Price`

In [ ]:
# YOUR CODE HERE
# Assuming your DataFrame is named df
df_drop_size = df[df['Size'] != 'Varies with device']
plt.figure(figsize=(10, 6))

# Extract the Size and Price columns
size = df_drop_size['Size']
price = df_drop_size['Price']

# Create a scatter plot
plt.scatter(size, price, alpha=0.5, color='blue')

plt.title('Scatterplot of Size vs Price')
plt.xlabel('Size')
plt.ylabel('Price')
plt.yscale('log')

# Define a formatting function for the y-axis labels
def price_formatter(x, pos):
    return f"${x:.0f}"

# Apply the formatting function to the y-axis labels
plt.gca().get_yaxis().set_major_formatter(ticker.FuncFormatter(price_formatter))

plt.show()

#df.cor to get degree of correlation

##### Exercise 2: Find the popular app categories based on rating and no. of installs

**Hint:** [df.groupby.agg()](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html); Taking the average rating could be another approach



In [ ]:
# YOUR CODE HERE
category_popularity = df.groupby('Category').agg({
    'Rating': 'mean',
    'Installs': 'sum'
}).reset_index()

# Calculate a combined metric based on the average rating and total installs
category_popularity['CombinedMetric'] = category_popularity['Rating'] * category_popularity['Installs']

# Sort categories based on popularity
sorted_categories = category_popularity.sort_values(by='CombinedMetric', ascending=False)['Category']

# Display the popular categories
print("Popular App Categories based on Rating and Installs:")
print(category_popularity.loc[category_popularity['Category'].isin(sorted_categories)])

##### Exercise 3: How many apps are produced in each year category-wise ?

  * Create a `Year` column by slicing the values of `Last Updated` column and find the Year with most no. of apps produced

    **For example**, slice the year `2017` from `February 8, 2017`

  * Find the categories which have a consistent rating in each year

      **Hint:** `sns.countplot`

In [ ]:
# YOUR CODE HERE
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

# Create a new 'Year' column by extracting the year
df['Year'] = df['Last Updated'].dt.year

# Count the number of apps produced in each year category-wise
apps_per_year = df.groupby(['Year', 'Category']).size().reset_index(name='Count')

# Find the year with the most number of apps produced
most_apps_year = apps_per_year.loc[apps_per_year['Count'].idxmax()]

print("Apps produced in each year category-wise:")
print(apps_per_year)

print("\nYear with the most number of apps produced:")
print(most_apps_year[['Year', 'Count']])

#add a timeseries line chart or a countplot

In [ ]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

# Create a new 'Year' column by extracting the year
df['Year'] = df['Last Updated'].dt.year

# Calculate the standard deviation of ratings for each category in each year
rating_std_by_category = df.groupby(['Year', 'Category'])['Rating'].std().reset_index()

# Calculate the mean standard deviation for each category across years
mean_std_by_category = rating_std_by_category.groupby('Category')['Rating'].mean().reset_index()

# Sort categories based on mean standard deviation in ascending order
consistent_categories = mean_std_by_category.sort_values(by='Rating', )

plt.figure(figsize=(12, 6))

# Create a bar plot to visualize mean standard deviation for each category
sns.barplot(x='Rating', y='Category', data=consistent_categories, palette='viridis')

# Set plot labels and title
plt.xlabel('Mean Standard Deviation of Ratings')
plt.ylabel('Category')
plt.title('Consistency of Ratings Across Categories')

# Show the plot
plt.show()
#sort it out by highest rating to lowest

##### Exercise 4: Identify the highest paid apps with a good rating

In [ ]:
# YOUR CODE HERE
df['Price'] = pd.to_numeric(df['Price'].replace('[\$,]', '', regex=True), errors='coerce')

# Filter for paid apps
paid_apps = df[df['Type'] == 'Paid']

# Set a threshold for a "good" rating (adjust as needed)
good_rating_threshold = 4.5

# Filter for paid apps with a good rating
good_paid_apps = paid_apps[paid_apps['Rating'] >= good_rating_threshold]

# Sort the good paid apps by rating in descending order
highest_paid_good_apps = good_paid_apps.sort_values(by='Rating', ascending=False)

# Display the highest paid apps with a good rating
print("Highest Paid Apps with a Good Rating:")
print(highest_paid_good_apps[['App', 'Rating', 'Price']])



##### Exercise 5: Are the top-rated apps genuine ? How about checking reviews count of top-rated apps ?

In [ ]:
# YOUR CODE HERE
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

# Set a threshold for a "top-rated" app (adjust as needed)
top_rated_threshold = 4.5

# Filter for top-rated apps
top_rated_apps = df[df['Rating'] >= top_rated_threshold]

# Sort the top-rated apps by reviews count in descending order
top_rated_apps = top_rated_apps.sort_values(by='Rating', ascending=False)

# Display the top-rated apps along with their ratings and reviews count
print("Top-Rated Apps with High Reviews Count:")
print(top_rated_apps[['App', 'Rating', 'Reviews']])

##### Exercise 6: If the number of reviews of an app is very low, what could be the reason for its top-rating ?

It could be a newly launched app which very few people have installed and whoever has installed has rated it high.

##### Exercise 7: What is the 95% confidence interval for the rating of apps in the Google Play Store?

In [ ]:
# YOUR CODE HERE
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Calculate mean and standard deviation
mean_rating = np.mean(df['Rating'])
std_dev_rating = np.std(df['Rating'], ddof=1)

# Calculate the 95% confidence interval
confidence_interval = stats.norm.interval(0.95, loc=mean_rating, scale=std_dev_rating/np.sqrt(len(df['Rating'])))

print("95% Confidence Interval for Rating:")
print(confidence_interval)

##### Exercise 8: Test if there is a statistically significant difference in the ratings between free and paid apps using a t-test

Steps:

* Set the null hypothesis and alternate hypothesis
* Separate the ratings of free and paid apps.
* Perform t-test: Use an independent samples t-test.
* Interpret results based on the p-value, decide whether to reject or fail to reject the null hypothesis.

**Null Hypothesis (H0):**
There is no significant difference in ratings between free and paid apps.

**Alternative Hypothesis (H1):**
There is a significant difference in ratings between free and paid apps.

In [ ]:
# YOUR CODE HERE
# Separate ratings of free and paid apps
ratings_free = df[df['Type'] == 'Free']['Rating']
ratings_paid = df[df['Type'] == 'Paid']['Rating']

# Perform independent samples t-test
t_statistic, p_value = ttest_ind(ratings_free, ratings_paid, equal_var=False)

# Set significance level (alpha)
alpha = 0.05

# Interpret the results
print("t-statistic:", t_statistic)
print("p-value:", p_value)

if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference in ratings between free and paid apps.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in ratings between free and paid apps.")